In [1]:
import json
import pandas as pd

In [56]:
# Read in the geojson file
data = None
with open("berney_divisions.geojson", "r") as f:
    data = json.load(f)

# Read website\public\data\bipartite_divisions_type_metiers.csv
df = pd.read_csv("bipartite_divisions_type_metiers.csv")

# Read data/1832_v4_preprocessed.csv
df_pop = pd.read_csv("../../../data/1832_v4_preprocessed.csv")

# Get the population of each division
population = df_pop.groupby("division").size().to_dict()

# Get the origin category for eachn division
origin = df_pop.groupby(["division", "chef_origine_class"]).count()["nom_rue"]

# In the properties section, add the values
# For example, for division 1:
# "properties": {"id": 1, "name": "marterey", "jobs": {"agricole": 82, "artisanat": 93,...
for feature in data["features"]:
    division = feature["properties"]["id"]
    # Add the population
    feature["properties"]["population"] = population[division]
    division_df = df.query("division == @division")
    jobs = {}
    for index, row in division_df.iterrows():
        jobs[row["job_category"]] = row["Weight"]
    feature["properties"]["jobs"] = jobs

# In the properties section, add the origins values from the origin dataframe
# For example, for division 1:
# "properties": {"id": 1, "name": "marterey", "origins": {"aigle": 1, moudon: 6, ...}
for feature in data["features"]:
    division = feature["properties"]["id"]
    origins = {}
    for index, row in origin[division].items():
        origins[index] = row
    feature["properties"]["origins"] = origins

In [57]:
df.job_category.unique()

array(['administration', 'agricole', 'artisanat', 'commerce',
       'construction', 'rente', 'service'], dtype=object)

In [86]:
df_pop.groupby(["division", "chef_origine_class"]).count()["nom_rue"].reset_index().groupby("chef_origine_class").max()["nom_rue"]

chef_origine_class
aigle                 8
angleterre           14
aubonne              10
avenches              3
cossonay             17
echallens            12
france               21
fribourg              6
geneve                6
grandson              2
italie                3
la_vallee            14
lausanne            149
lavaux               53
morges               26
moudon               13
neuchatel             3
nyon                 10
orbe                 10
oron                 15
payerne               4
pays_d_enhaut        11
rolle                 8
suisse_allemande     36
vaud                  6
vevey                15
yverdon              12
Name: nom_rue, dtype: int64

In [74]:
# 
df_pop.groupby(["division", "chef_origine_class"]).count()["nom_rue"].min()

1

In [59]:
# Save the new geojson file
with open("berney_divisions.geojson", "w") as f:
    json.dump(data, f)